In [ ]:
import cv2
import os

# Initialize the camera
cap = cv2.VideoCapture(0)

# Define the box coordinates
box_start = (150, 100)  # Top-left corner of the box
box_end = (400, 300)    # Bottom-right corner of the box

# Initialize image counter
img_counter = 1

# Specify the folder where images will be saved
save_folder = r'Captured_Images_Folder"

# Create the folder if it doesn't exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Draw the box on the frame
    cv2.rectangle(frame, box_start, box_end, (0, 255, 0), 2)
    cv2.putText(frame, "Press 'c' to capture, 'q' to quit", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame with the box
    cv2.imshow("Camera", frame)

    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):  # Press 'c' to capture the image inside the box
        # Crop the image inside the box
        cropped_image = frame[box_start[1]:box_end[1], box_start[0]:box_end[0]]
        
        # Save the captured image in the specified folder with a unique filename
        filename = os.path.join(save_folder, f"captured_image_{img_counter}.jpg")
        cv2.imwrite(filename, cropped_image)
        print(f"Image saved as '{filename}'")
        img_counter += 1  # Increment the counter for the next capture

    elif key == ord('q'):  # Press 'q' to quit
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


Image saved as 'C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\pipeline\model 1\captured images\captured_image_1.jpg'


Segmentation Model (It segments the eye conjunctiva part from the real time image)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def load_model(model_path):
    """Load a pre-trained U-Net model."""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")
    return tf.keras.models.load_model(model_path, compile=False)

model_path = r'Segmentation_Model_Path'
model = load_model(model_path)
h = 256  # Image height
w = 256  # Image width

def predict_mask(model, eye_image):
    """Predict the mask for the given eye image."""
    img_input = np.expand_dims(eye_image, axis=0)
    print(f"Input shape for model: {np.shape(img_input)}")
    mask = model.predict(img_input)
    return mask

# Folder path containing eye images
folder_path = r'Captured_Images_Path'
output_folder = r'Segmented_Images_Path'

# Process each image in the folder
for image_name in os.listdir(folder_path):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Ensure valid image formats
        image_path = os.path.join(folder_path, image_name)

        # Preprocess the image
        eye_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        eye_image = cv2.resize(eye_image, (w, h))
        eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        eye_image = eye_image / 255.0  # Normalize to [0, 1]

        # Predict the mask
        mask = predict_mask(model, eye_image)
        mask = mask * 255  # Scale mask values to [0, 255]
        mask = np.squeeze(mask)  # Remove singleton dimensions

        # Apply thresholding
        ret, th1 = cv2.threshold(mask, 0.025 * 255, 255, cv2.THRESH_BINARY)
        th1 = np.expand_dims(th1, axis=-1)  # Add back the channel dimension
        th1 = th1 / 255.0  # Normalize thresholded mask

        # Crop the eye image using the thresholded mask
        cropped_eye_img = th1 * eye_image

        # Scale the cropped image back to [0, 255] for saving
        cropped_eye_img = (cropped_eye_img * 255).astype(np.uint8)

        # Convert the cropped image back to BGR for saving with OpenCV
        cropped_eye_img_bgr = cv2.cvtColor(cropped_eye_img, cv2.COLOR_RGB2BGR)

        # Save extracted conjunctiva image
        save_path = os.path.join(output_folder, f"segmented_{image_name}")
        cv2.imwrite(save_path, cropped_eye_img_bgr)

        print(f"Saved segmented image: {save_path}")

        # Display the cropped eye image
        # plt.imshow(cropped_eye_img)  # Display in RGB format
        # plt.title(f"Cropped Eye Image for {image_name}")
        # # plt.show()

Input shape for model: (1, 256, 256, 3)
1/1 [==============================] - 1s 805ms/step
Saved segmented image: C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\pipeline\model 1\segmented images\segmented_1683272457169.jpg
Input shape for model: (1, 256, 256, 3)
1/1 [==============================] - 0s 79ms/step
Saved segmented image: C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\pipeline\model 1\segmented images\segmented_1683518265539.jpg
Input shape for model: (1, 256, 256, 3)
1/1 [==============================] - 0s 61ms/step
Saved segmented image: C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\pipeline\model 1\segmented images\segmented_1684219703477.jpg
Input shape for model: (1, 256, 256, 3)
1/1 [==============================] - 0s 62ms/step
Saved segmented image: C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\pipeline\model 1\segmented images\segmented_a.png


Classification Model ( It classifies the anemia condition and severity )

In [ ]:
import os
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

# Define the preprocess_image function
def preprocess_image(img_path, target_size=(224, 224)):
    
    # Load the image and resize it
    img = image.load_img(img_path, target_size=target_size)
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the input shape (batch_size, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image using MobileNetV3's preprocessing function
    img_array = preprocess_input(img_array)
    
    return img_array

# Load the model
# model = load_model(r'C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\classification model\model.h5')
model = load_model(r'Classification_Model_Path')

# Define class labels (replace with your actual class names)
class_labels = ['Mild Anemia', 'Moderate Anemia', 'No Anemia', 'Severe Anemia']

# Path to the folder containing images
folder_path = r'Segmented_Images_Path'

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can add more extensions if needed)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Full path to the image
        img_path = os.path.join(folder_path, filename)
        
        # Preprocess the image
        sample_image = preprocess_image(img_path)
        
        # Make a prediction
        predictions = model.predict(sample_image)
        
        # Get the predicted class index
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        
        # Map the class index to the class name
        predicted_class_name = class_labels[predicted_class_index]
        if predicted_class_name == "Mild Anemia":
            predicted_class_name = "No Anemia"
        # Print the result
        print(f"Image: {filename} -> Predicted class: {predicted_class_name}")

1/1 [==============================] - 1s 789ms/step
Image: segmented_1683272457169.jpg -> Predicted class: No Anemia
1/1 [==============================] - 0s 33ms/step
Image: segmented_1683518265539.jpg -> Predicted class: Moderate Anemia
1/1 [==============================] - 0s 30ms/step
Image: segmented_1684219703477.jpg -> Predicted class: Moderate Anemia
1/1 [==============================] - 0s 32ms/step
Image: segmented_a.png -> Predicted class: Moderate Anemia
